In [1]:
from keras.utils.np_utils import to_categorical
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

In [2]:
batch_size = 64
num_classes = 10
epochs = 120

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

input_shape = x_train.shape[1:]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [5]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=batch_size)

In [6]:

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# compile model
opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


callbacks = [
    #EarlyStopping(monitor='loss', patience=3),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
]

steps = int(x_train.shape[0] / batch_size)
model.fit(it_train, steps_per_epoch=steps, epochs=epochs, validation_data=(x_test, y_test), verbose=1, callbacks=callbacks)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/120
781/781 [==============================] - 49s 45ms/step - loss: 1.8298 - accuracy: 0.4029 - val_loss: 1.6702 - val_accuracy: 0.4296 - lr: 0.0100
Epoch 2/120
781/781 [==============================] - 36s 46ms/step - loss: 1.0965 - accuracy: 0.6102 - val_loss: 1.1094 - val_accuracy: 0.6352 - lr: 0.0100
Epoch 3/120
781/781 [==============================] - 36s 46ms/step - loss: 0.9199 - accuracy: 0.6791 - val_loss: 1.0632 - val_accuracy: 0.6570 - lr: 0.0100
Epoch 4/120
781/781 [==============================] - 36s 46ms/step - loss: 0.7898 - accuracy: 0.7269 - val_loss: 0.9471 - val_accuracy: 0.6836 - lr: 0.0100
Epoch 5/120
781/781 [==============================] - 35s 45ms/step - loss: 0.7248 - accuracy: 0.7514 - val_loss: 0.7249 - val_accuracy: 0.7531 - lr: 0.0100
Epoch 6/120
781/781 [==============================] - 35s 45ms/step - loss: 0.6722 - accuracy: 0.7701 - val_loss: 0.7814 - val_accuracy: 0.7356 - lr: 0.0100
Epoch 7/120
781/781 [==============================]